In [1]:
import pandas as pd
from numpy import nan 

In [23]:
df1 = pd.DataFrame(data={
    'segmenter':[nan,'alpha','alpha','alpha','alpha','omega','omega','omega','omega','omega'],
                         'lookup1':[nan,nan,'c','d','e','a',nan,'c','d','e'],
                         'lookup2':[nan,nan,'banana','pear','pineapple','apple',nan,'banana','pear','pineapple'],
                         'custs':[1,2,3,4,5,1,2,3,4,5]})
df2 = pd.DataFrame(data={'lookup1':['a',nan,'i','o','e'],'lookup2':['apple',nan,'hyena','pear','pineapple'],
                         'custs':[2,4,6,8,10]})

In [10]:
def divider(df, segmenter):

    """
    Divides the last column of a row in a df by the sum of all the other rows that have the same combination of NaN/not NaN
    in their other columns. This is designed for profiling - if you have two columns, one showing one profile, and one
    showing another, rows will only be divided by the other relevant rows
    """
        
    columns = list(df.columns)[:-1]
    
    df.dropna(axis = 0, how = 'all', subset = columns, inplace = True)
    
    out_df = pd.DataFrame()
    
    if segmenter: # if this table is duplicated over different segments, we need to calculate %s within segment
    
        for segment in df[columns[0]].unique():
            
            for column in columns[1:]:

                    temp_df = df[(df[column].isnull() == False) & (df[columns[0]] == segment)].copy()
                    temp_df['per'] = temp_df.iloc[:,-1] / temp_df.iloc[:,-1].sum()

                    out_df = out_df.append(temp_df, ignore_index = True)
                    
    else:
        
        for column in columns:

            temp_df = df[df[column].isnull() == False].copy()
            temp_df['per'] = temp_df.iloc[:,-1] / temp_df.iloc[:,-1].sum()

            out_df = out_df.append(temp_df, ignore_index = True)
        
    return out_df

In [75]:
# identifies the number of columns in the dataframe, ignoring the first one if the argument 'segmenter' is true
def indexer(df1,df2,segmenter=False, use_divider = True):
    '''
    divides the values in the last column of df1 by the values in the last column in df2, joining on all the other columns in 
    the table (nulls will be joined to nulls). All columns need to match to calculate the index, otherwise NaN is returned
    
    Arguments:
        df1 - the numerator table for your index calculation.
        df2 - the denominator table for your index calculation.
        segmenter - indicates whether the first column is showing levels in a segmentation, and therefore whether
                    the first column should be included when looking for what to join on.
        use_divider - whether to call the divider function to get row %s, or not - if you already have them.
    '''
    
    if use_divider:
        
        numerator = divider(df1, segmenter)
        denominator = divider(df2, False)
                              
    else:
        numerator = df1.copy()
        denominator = df2.copy()
                     
    # joins two dfs together - now the last two columns will be the columns to be divided
    if segmenter:
        column_list = list(df1.columns[1:-1])
        result_df = numerator.merge(denominator, how='left', left_on = column_list, right_on = column_list)
    else:
        column_list = list(df1.columns[:-1])
        result_df = numerator.merge(denominator,how='left',left_on = column_list, right_on = column_list)
                
    # calculates index by dividing the second last column by the last column
    result_df['index'] = result_df[result_df.columns[-3]]/result_df[result_df.columns[-1]]
    
    # retains just the columns we're interested in
    if segmenter:
        column_list.insert(0,df1.columns[0])
    column_list.append('index')

    result_df = result_df[[i for i in column_list]]
    
    return result_df